In [15]:
import torch
import torch.nn as nn

In [3]:
input_embeddings = torch.tensor([
    [0.43, 0.15, 0.89], # Your    -> x_0
    [0.55, 0.87, 0.66], # journey -> x_1
    [0.57, 0.85, 0.64], # starts  -> x_2
    [0.22, 0.58, 0.33], # with    -> x_3
    [0.77, 0.25, 0.10], # one     -> x_4
    [0.05, 0.80, 0.55], # step    -> x_5
])

## define the parameters, project the inputs

In [4]:
d_in = input_embeddings.shape[1]
d_out = 2
W_query = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

In [6]:
queries = input_embeddings @ W_query
keys = input_embeddings @ W_key
values = input_embeddings @ W_value

## calculate attention scores

In [8]:
attn_scores = queries @ keys.T
display(attn_scores.shape)
display(attn_scores)

torch.Size([6, 6])

tensor([[-0.5173, -0.2734, -0.2802, -0.0701, -0.3233, -0.0188],
        [ 0.8422,  0.3349,  0.3169,  0.1276, -0.0961,  0.3178],
        [ 0.7687,  0.3026,  0.2854,  0.1168, -0.1047,  0.2979],
        [ 0.8220,  0.3462,  0.3338,  0.1221,  0.0158,  0.2596],
        [-0.7745, -0.3645, -0.3628, -0.1104, -0.2306, -0.1451],
        [ 1.5474,  0.6628,  0.6423,  0.2286,  0.0922,  0.4599]])

## mask attention scores, weigths and context vetors

In [9]:
context_length = attn_scores.shape[0]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
display(mask)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [10]:
masked_attn_scores = attn_scores.masked_fill(mask.bool(), -torch.inf)
display(masked_attn_scores)

tensor([[-0.5173,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.8422,  0.3349,    -inf,    -inf,    -inf,    -inf],
        [ 0.7687,  0.3026,  0.2854,    -inf,    -inf,    -inf],
        [ 0.8220,  0.3462,  0.3338,  0.1221,    -inf,    -inf],
        [-0.7745, -0.3645, -0.3628, -0.1104, -0.2306,    -inf],
        [ 1.5474,  0.6628,  0.6423,  0.2286,  0.0922,  0.4599]])

In [11]:
masked_attn_weights = torch.softmax(masked_attn_scores / keys.shape[-1]**0.5, dim=-1)
display(masked_attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5887, 0.4113, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4116, 0.2960, 0.2924, 0.0000, 0.0000, 0.0000],
        [0.3298, 0.2356, 0.2335, 0.2011, 0.0000, 0.0000],
        [0.1483, 0.1982, 0.1984, 0.2372, 0.2179, 0.0000],
        [0.3052, 0.1633, 0.1609, 0.1201, 0.1091, 0.1415]])

In [13]:
context_vec = masked_attn_weights @ values
display(context_vec)

tensor([[ 3.5024,  0.3855],
        [ 2.7445,  0.1736],
        [ 2.4217,  0.0717],
        [ 2.0268,  0.0497],
        [ 1.5985, -0.1782],
        [ 1.8771,  0.0241]])

## as a single class + dropout + work on batches

In [16]:
class CausalAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out)
        self.W_key = nn.Linear(d_in, d_out)
        self.W_value = nn.Linear(d_in, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
    
    def forward(self, x):
        n_batches, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # we additionally add dropout here; this is usual place to do this
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

In [18]:
input_embeddings_batch = torch.stack((input_embeddings, input_embeddings), dim=0)
input_embeddings_batch.shape

torch.Size([2, 6, 3])

In [20]:
torch.manual_seed(123)
context_length = input_embeddings_batch.shape[1]
c_attn = CausalAttention(d_in, d_out, context_length, dropout=0.0)
context_vecs = c_attn(input_embeddings_batch)
display(context_vecs.shape)
display(context_vecs)

torch.Size([2, 6, 2])

tensor([[[-0.0960,  0.7940],
         [ 0.0285,  0.9387],
         [ 0.0657,  0.9850],
         [ 0.1062,  0.9604],
         [ 0.0659,  0.9308],
         [ 0.1188,  0.9375]],

        [[-0.0960,  0.7940],
         [ 0.0285,  0.9387],
         [ 0.0657,  0.9850],
         [ 0.1062,  0.9604],
         [ 0.0659,  0.9308],
         [ 0.1188,  0.9375]]], grad_fn=<UnsafeViewBackward0>)